In [1]:
!gdown --id 1qiUDDoYyRLBiKOoYWdFl_5WByHE8Cugu

C:\Users\Administrator\anaconda3\envs\aio2024_env\Lib\site-packages\gdown\__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1qiUDDoYyRLBiKOoYWdFl_5WByHE8Cugu
To: d:\AIO-module-5\MLP\Auto_MPG_data.csv

  0%|          | 0.00/15.4k [00:00<?, ?B/s]
100%|██████████| 15.4k/15.4k [00:00<00:00, 7.73MB/s]


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
random_state = 59
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_state)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available()else 'cpu')


In [6]:
dataset_path = 'Auto_MPG_data.csv'
dataset = pd.read_csv(dataset_path)

In [ ]:
X = dataset.drop(columns ='MPG').values
y = dataset['MPG'].values

In [ ]:
val_size = 0.2
test_size = 0.125
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
X, y,
test_size = val_size,
random_state = random_state,
shuffle = is_shuffle
)

X_train, X_test, y_train, y_test = train_test_split(
X_train, y_train,
test_size = test_size,
random_state = random_state,
shuffle = is_shuffle
)

In [ ]:
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

X_train = torch.tensor(X_train, dtype = torch.float32)
X_val = torch.tensor(X_val, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.float32)
y_val = torch.tensor(y_val, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.float32)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[ idx ], self.y[ idx ]

In [ ]:
batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
train_loader = DataLoader(train_dataset,batch_size = batch_size,shuffle = True)
val_loader = DataLoader(val_dataset,batch_size = batch_size,shuffle = False)

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.linear2 = nn.Linear(hidden_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        out = self.output(x)
        return out.squeeze(1)

In [ ]:
input_dims = X_train.shape [1]
output_dims = 1
hidden_dims = 64

model = MLP(input_dims= input_dims,hidden_dims= hidden_dims,output_dims= output_dims).to(device)

In [ ]:
lr = 1e-2
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [16]:
def r_squared(y_true , y_pred):
    y_true = torch.Tensor(y_true).to(device)
    y_pred = torch.Tensor(y_pred).to(device)
    mean_true = torch.mean(y_true)
    ss_tot = torch.sum((y_true - mean_true)** 2)
    ss_res = torch.sum((y_true - y_pred)** 2)
    r2 = 1 -(ss_res / ss_tot)
    return r2

In [17]:
epochs = 100
train_losses = []
val_losses = []
train_r2 = []
val_r2 = []
for epoch in range(epochs):
    train_loss = 0.0
    train_target = []
    val_target = []
    train_predict = []
    val_predict = []
    model.train()
    for X_samples , y_samples in train_loader :
        X_samples = X_samples .to(device)
        y_samples = y_samples .to(device)
        optimizer.zero_grad()
        outputs = model(X_samples)
        train_predict += outputs.tolist()
        train_target += y_samples.tolist()
        loss = criterion(outputs , y_samples)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_r2.append(r_squared(train_target , train_predict))
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_samples , y_samples in val_loader :
            X_samples = X_samples .to(device)
            y_samples = y_samples .to(device)
            outputs = model(X_samples)
            val_predict += outputs.tolist()
            val_target += y_samples.tolist()
            loss = criterion(outputs , y_samples)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_r2.append(r_squared(val_target , val_predict))
    print(f'\nEPOCH { epoch + 1}:\ tTraining loss : { train_loss:.3f}\
    tValidation loss : {val_loss:.3f}')


EPOCH 1:\ tTraining loss : 282.770    tValidation loss : 88.671

EPOCH 2:\ tTraining loss : 137.640    tValidation loss : 72.293

EPOCH 3:\ tTraining loss : 71.207    tValidation loss : 19.668

EPOCH 4:\ tTraining loss : 24.286    tValidation loss : 166.905

EPOCH 5:\ tTraining loss : 89.477    tValidation loss : 19.816

EPOCH 6:\ tTraining loss : 17.982    tValidation loss : 17.116

EPOCH 7:\ tTraining loss : 20.667    tValidation loss : 8.669

EPOCH 8:\ tTraining loss : 38.545    tValidation loss : 36.080

EPOCH 9:\ tTraining loss : 20.425    tValidation loss : 30.156

EPOCH 10:\ tTraining loss : 20.308    tValidation loss : 49.483

EPOCH 11:\ tTraining loss : 33.849    tValidation loss : 21.479

EPOCH 12:\ tTraining loss : 10.136    tValidation loss : 5.739

EPOCH 13:\ tTraining loss : 15.319    tValidation loss : 15.422

EPOCH 14:\ tTraining loss : 12.338    tValidation loss : 13.156

EPOCH 15:\ tTraining loss : 14.816    tValidation loss : 8.520

EPOCH 16:\ tTraining loss : 11.11

In [20]:
model.eval ()
with torch.no_grad () :
    y_hat = model(X_test.to( device ))
    test_set_r2 = r_squared (y_hat , y_test)
    print ('Evaluation on test set :')
    print (f'R2: {test_set_r2}')

Evaluation on test set :
R2: 0.8407537937164307
